In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

100%|██████████| 21600/21600 [00:00<00:00, 25981.52it/s]


In [3]:
DF.cpu.unique()

array(['Intel(R) Xeon(R) CPU E5-2695 v4 @ 2.10GHz',
       'Intel(R) Xeon(R) Platinum 8352V CPU @ 2.10GHz',
       'Intel(R) Xeon(R) CPU E5-2695 v3 @ 2.30GHz',
       'AMD EPYC 7453 28-Core Processor',
       'Intel(R) Xeon(R) CPU E5-2699 v3 @ 2.30GHz',
       'AMD EPYC 9654 96-Core Processor', ''], dtype=object)

In [4]:
COLUMNS = [
    # "data_pruning",
    # "facts",
    "optimiser",
    # "features",
    "iterations",
    "feature_pruning",
]

def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "fd"]
    df = df[df.features == "wl"]
    df = df[df.data_pruning == "equivalent-weighted"]
    # df = df[df.feature_pruning.isin({"none", "collapse-all"})]
    df = df[df.feature_pruning != "collapse-all"]
    return df

In [5]:
dfs = {}
for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    # df = df.fillna(0)
    # replace 0 with NaN
    # df = df.replace(0, -1)

    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)
    dfs[metric] = df

solved


domain                                   blocksworld  childsnack  ferry  \
optimiser iterations feature_pruning                                      
rank-svm  2          0none                        75          24     68   
                     1collapse-all-x              71          29     69   
                     2collapse-layer-x            45          31     70   
                     3collapse-layer-y            44          30     70   
                     4collapse-layer-f            71          26     67   
                     5collapse-layer-yf           45          31     69   
          4          0none                        65          32     68   
                     1collapse-all-x              66          37     66   
                     2collapse-layer-x            45          32     68   
                     3collapse-layer-y            46          31     70   
                     4collapse-layer-f            71          36     66   
                     5collapse-layer-yf           44          31     69   
svr       2          0none                        32          18     70   
                     1collapse-all-x              31          18     69   
                     2collapse-layer-x            45          22     67   
                     3collapse-layer-y            45          20     66   
                     4collapse-layer-f            33          20     67   
                     5collapse-layer-yf           43          20     66   
          4          0none                        62          30     67   
                     1collapse-all-x              53          18     67   
                     2collapse-layer-x            46          22     68   
                     3collapse-layer-y            45          20     69   
                     4collapse-layer-f            38          20     66   
                     5collapse-layer-yf           42          20     67   

domain                                   floortile  miconic  rovers  \
optimiser iterations feature_pruning                                  
rank-svm  2          0none                       2       90      39   
                     1collapse-all-x             2       90      41   
                     2collapse-layer-x           3       90      41   
                     3collapse-layer-y           3       90      41   
                     4collapse-layer-f           2       90      40   
                     5collapse-layer-yf          3       90      43   
          4          0none                       2       87       0   
                     1collapse-all-x             2       90      42   
                     2collapse-layer-x           3       90      41   
                     3collapse-layer-y           3       90      42   
                     4collapse-layer-f           2       88      41   
                     5collapse-layer-yf          3       90      43   
svr       2          0none                       2       90      40   
                     1collapse-all-x             2       90      46   
                     2collapse-layer-x           2       90      43   
                     3collapse-layer-y           2       90      44   
                     4collapse-layer-f           1       90      37   
                     5collapse-layer-yf          1       90      43   
          4          0none                       2       87      35   
                     1collapse-all-x             2       90      38   
                     2collapse-layer-x           2       90      43   
                     3collapse-layer-y           2       90      43   
                     4collapse-layer-f           1       89      34   
                     5collapse-layer-yf          1       90      44   

domain                                   satellite  sokoban  spanner  \
optimiser iterations feature_pruning                                   
rank-svm  2          0none                      32       3

In [6]:
for optimisation, iterations in product(["svr", "rank-svm"], ["2", "4"]):
    print(optimisation, iterations)
    df = get_df_copy()
    df = df[df.optimiser == optimisation]
    df = df[df.iterations == iterations]
    df = df[df.solved]
    df.solved = df.solved.astype(int)
    
    fig = px.ecdf(df, x="runtime", color="feature_pruning", ecdfnorm=None, log_x=True)
    # hide axis labels
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')
    # size
    fig.update_layout(width=600, height=400)
    # remove margins
    fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    # font size
    fig.update_layout(font=dict(size=20))
    # hide legend
    # fig.update_layout(showlegend=False)

    fig.show()

svr 2


svr 4


rank-svm 2


rank-svm 4


In [7]:
df = dfs["solved"]
latex = df.to_latex()
latex = latex.replace("_", " ")
for domain in DOMAINS:
    latex = latex.replace(domain, '\\' + domain[:2] + "{}")
latex = latex.replace("total", r"$\Sigma$")
latex = latex.replace("domain", "")
for k, v in FEATURE_GENERATION_PREFIX.items():
    to_replace = f"{v}{k}"
    latex = latex.replace(to_replace, f"\\{k}{{}}".replace("-", ""))
latex = latex.replace(r"\begin{tabular}", r"\begin{tabularx}{\textwidth}")
latex = latex.replace(r"rrrrrrrrrrr}", r"Y Y Y Y Y Y Y Y Y Y Y}")
latex = latex.replace(r"\end{tabular}", r"\end{tabularx}")
latex = latex.replace(r"\cline{1-14} \cline{2-14}", r"\midrule")
latex = latex.replace(r"\midrule" + "\n" + r"\bottomrule", r"\bottomrule")
latex = latex.replace(r"\cline", r"\cmidrule")
with open("/home/dzc/manuscripts/socs-25-redundant/tables/coverage.tex", "w") as f:
    f.write(latex)

In [24]:
df = dfs["solved"]
df = df.drop(columns=DOMAINS)
latex = df.to_latex()
latex = latex.replace("_", " ")
for domain in DOMAINS:
    latex = latex.replace(domain, '\\' + domain[:2] + "{}")
latex = latex.replace("total", r"$\Sigma$")
latex = latex.replace("domain", "")
for k, v in FEATURE_GENERATION_PREFIX.items():
    to_replace = f"{v}{k}"
    latex = latex.replace(to_replace, f"\\{k}{{}}".replace("-", ""))
latex = latex.replace(r"\begin{tabular}", r"\begin{tabularx}{\columnwidth}")
latex = latex.replace(r"\end{tabular}", r"\end{tabularx}")
latex = latex.replace(r"\midrule" + "\n" + r"\bottomrule", r"\bottomrule")
with open("/home/dzc/manuscripts/socs-25-redundant/tables/coverage_short.tex", "w") as f:
    f.write(latex)

In [8]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        # print(len(df))
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"] + " - " + df["data_pruning"] # + " - " + df["feature_pruning"]
        # df["config"] = " _ ".join([df[f] for f in ["features", "iterations", "optimiser", "data_pruning", "feature_pruning"]])
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="feature_pruning",
            line_dash="feature_pruning",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        fig.update_xaxes(categoryorder='array', categoryarray=PROBLEMS)
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)